## FOCUSED
- Author:  Reshama Shaikh

## Data Sources
1. S&P 500
1. OSCI Index
1. Suzy spreadsheet
1. OSPO Landscape (Todo group)

---

## Date

In [1]:
from datetime import date

current_date = date.today()
print("Today's date:", current_date)

Today's date: 2022-05-31


## Import libraries


In [2]:
import pandas as pd 
import numpy as np
import requests
import time
import csv
from pprint import pprint

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
%load_ext watermark
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.25.0

Compiler    : Clang 11.1.0 
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Git hash: d96b799f1be6702f10c9dfd1db2e11b78710db35

pandas  : 1.3.3
csv     : 1.0
plotly  : 5.4.0
requests: 2.25.1
numpy   : 1.20.2



## Set pandas view options

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Data Source:  S&P 500

In [6]:
datapath="data_raw/"

def get_raw_data(data_url, file_name):
    file_url = data_url + "/" + file_name
    !wget -N {file_url} -P {datapath} 
    
def read_raw_data(file_name):
    df_use = pd.read_csv(datapath+file_name)
    
    return df_use

In [7]:
data_url='https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/'
file_name = 'constituents.csv'

#run once
#get_raw_data(data_url, file_name)

In [8]:
# run once
#!mv data_raw/constituents.csv data_raw/sp500.csv 

In [9]:
file_name='sp500.csv'
df_sp = read_raw_data(file_name)

print(df_sp.shape)
df_sp.head(5)

(505, 3)


,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [10]:
df_sp = df_sp.rename(columns=
                         {'Name': 'company',
                          'Symbol': 'symbol',
                          'Sector': 'sector',
                          },)

df_sp['in_sp500']=1
df_sp['country'] = "United States"


print(df_sp.shape)
df_sp.head(5)

(505, 5)


,symbol,company,sector,in_sp500,country
0,MMM,3M,Industrials,1,United States
1,AOS,A. O. Smith,Industrials,1,United States
2,ABT,Abbott Laboratories,Health Care,1,United States
3,ABBV,AbbVie,Health Care,1,United States
4,ABMD,Abiomed,Health Care,1,United States


---

## 2. Data Source:  OSCI

In [11]:
data_url='https://ststaticprodosciwebz2vmu.blob.core.windows.net/data/osci-ranking/daily'
file_name = '2022-04-17.json'

#run once
#get_raw_data(data_url, file_name)

In [12]:
# run once
#!mv data_raw/2022-04-17.json data_raw/osci.json 

In [13]:
file_name='osci.json'
file_name

'osci.json'

In [14]:
import json
 
# Opening JSON file
f = open('data_raw/osci.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

print(len(data))
#print(data)


# Iterating through the json
# list
for i in data['date'][0]:
    print(i)
 
# Closing file
f.close()

3
2


In [15]:
# print(data['date'])
# print(type(data))
# print(type(data['date']))
# #print(data['date'].['comparedDate'])
# #print(data['date'][0]['data'])
# print(data['date'], data['data'][0]['company'])

In [16]:
#pd.read_json('data_raw/osci.json')

In [17]:
# Normalizing data
multiple_level_data = pd.json_normalize(data, record_path =['data'])
#, meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], 
#meta_prefix='config_params_', record_prefix='dbscan_')

# Saving to CSV format
#multiple_level_data.to_csv('multiplelevel_normalized_data.csv', index=False)

multiple_level_data.head(1)

,positionChange,company,activeContributors,activeContributorsChange,totalCommunity,totalCommunityChange,position,yoy,contributors,languages,licenses,industry
0,0.0,Google,4017.0,273.0,8676,431.0,1,"[{'date': '2020-04-17', 'active': 2587.0}, {'d...","[{'Contributor': 'Modular Magician', 'Commits'...","[{'name': 'Brightscript', 'amount': 5}, {'name...","[{'name': 'apache-2.0', 'amount': 180027}, {'n...",Technology


In [18]:
keep_cols = ['positionChange', 'company', 'activeContributors', 'activeContributorsChange', 
             'totalCommunity', 'totalCommunityChange', 'position', 'industry'
            ]

df_osci = multiple_level_data.filter(keep_cols)
df_osci['in_osci']=1

df_osci = df_osci.rename(columns=
                         {'position': 'osci_position',
                          'industry': 'osci_industry'
                             })

print(df_osci.shape)
df_osci.head(5)

(298, 9)


,positionChange,company,activeContributors,activeContributorsChange,totalCommunity,totalCommunityChange,osci_position,osci_industry,in_osci
0,0.0,Google,4017.0,273.0,8676,431.0,1,Technology,1
1,0.0,Microsoft,3630.0,308.0,8128,545.0,2,Technology,1
2,0.0,Red Hat,2965.0,159.0,4594,188.0,3,Technology,1
3,0.0,Intel,1557.0,112.0,3504,190.0,4,Technology,1
4,0.0,IBM,1419.0,128.0,3515,259.0,5,Technology,1


## Merge: S&P 500 + OSCI

In [19]:
print(df_osci.shape)
print(len(df_osci['company'].unique()))

print(df_sp.shape)
print(len(df_sp['company'].unique()))

(298, 9)
298
(505, 5)
505


In [20]:
df_parta = df_sp.merge(df_osci, left_on="company", right_on="company", how="outer")
print(len(df_parta))
df_parta.head()

781


,symbol,company,sector,in_sp500,country,positionChange,activeContributors,activeContributorsChange,totalCommunity,totalCommunityChange,osci_position,osci_industry,in_osci
0,MMM,3M,Industrials,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOS,A. O. Smith,Industrials,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABT,Abbott Laboratories,Health Care,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBV,AbbVie,Health Care,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABMD,Abiomed,Health Care,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(len(df_parta['company'].unique()))

781


In [22]:
df_parta = df_parta.rename(columns=
                         {'position': 'osci_position'
                             },)

new_order = ['symbol', 'company','sector', 'country','osci_industry',
             'in_sp500','in_osci', 'osci_position'
            ]

df_parta = df_parta.reindex(new_order, axis=1)

In [23]:
df_parta.head()

,symbol,company,sector,country,osci_industry,in_sp500,in_osci,osci_position
0,MMM,3M,Industrials,United States,NaN,1.0,NaN,NaN
1,AOS,A. O. Smith,Industrials,United States,NaN,1.0,NaN,NaN
2,ABT,Abbott Laboratories,Health Care,United States,NaN,1.0,NaN,NaN
3,ABBV,AbbVie,Health Care,United States,NaN,1.0,NaN,NaN
4,ABMD,Abiomed,Health Care,United States,NaN,1.0,NaN,NaN


In [24]:
df_check = df_parta.groupby(['in_sp500','in_osci']).count()
df_check

,,symbol,company,sector,country,osci_industry,osci_position
in_sp500,in_osci,,,,,,
1.0,1.0,22,22,22,22,22,22


In [25]:
df_check = df_parta.groupby(['in_sp500']).count()
df_check

,symbol,company,sector,country,osci_industry,in_osci,osci_position
in_sp500,,,,,,,
1.0,505,505,505,505,22,22,22


In [26]:
df_check = df_parta.groupby(['in_osci']).count()
df_check

,symbol,company,sector,country,osci_industry,in_sp500,osci_position
in_osci,,,,,,,
1.0,22,298,22,22,298,22,298


In [27]:
df_check = df_parta['company'].count() 
df_check

781

In [28]:
#df_parta.head(5)

In [29]:
#df_parta.dtypes

In [30]:
#df_sample = df_parta.sample(20)

In [31]:
#df_sample.sort_values(by=['company'])

In [32]:
df_parta[df_parta['company']=="Google"]

,symbol,company,sector,country,osci_industry,in_sp500,in_osci,osci_position
505,NaN,Google,NaN,NaN,Technology,NaN,1.0,1.0


In [33]:
df_sorted = df_parta.sort_values(by=['company'])
df_sorted.head(5)

,symbol,company,sector,country,osci_industry,in_sp500,in_osci,osci_position
0,MMM,3M,Industrials,United States,NaN,1.0,NaN,NaN
767,NaN,4teamwork,NaN,NaN,Technology,NaN,1.0,285.0
758,NaN,5minds,NaN,NaN,Technology,NaN,1.0,276.0
1,AOS,A. O. Smith,Industrials,United States,NaN,1.0,NaN,NaN
7,ADM,ADM,Consumer Staples,United States,NaN,1.0,NaN,NaN


In [34]:
df_sample = df_sorted.sample(10)
df_sample

,symbol,company,sector,country,osci_industry,in_sp500,in_osci,osci_position
727,NaN,Loodos Tech,NaN,NaN,Technology,NaN,1.0,244.0
572,NaN,Twilio,NaN,NaN,Technology,NaN,1.0,85.0
239,HII,Huntington Ingalls Industries,Industrials,United States,NaN,1.0,NaN,NaN
632,NaN,Adguard,NaN,NaN,Technology,NaN,1.0,149.0
763,NaN,Renovateapp,NaN,NaN,Technology,NaN,1.0,281.0
218,GPN,Global Payments,Information Technology,United States,NaN,1.0,NaN,NaN
672,NaN,Block.one,NaN,NaN,Technology,NaN,1.0,189.0
516,NaN,EPAM,NaN,NaN,Technology,NaN,1.0,19.0
612,NaN,Arbisoft,NaN,NaN,Technology,NaN,1.0,129.0
740,NaN,JankariTech,NaN,NaN,Technology,NaN,1.0,258.0


## 3. Data Source:  Suzy spreadsheet

In [35]:
file_name = 'classification_suzy.csv'

df_suzy = pd.read_csv('data_raw/'+file_name)
df_suzy['in_suzy']=1

df_suzy = df_suzy.rename(columns=
                         {'Company': 'company',
                          'Level of Engagement': 'os_engagement',
                          'TODO Member': 'todo_member',
                          'TODO Adopter': 'todo_adopter',
                          'Main Business': 'area',
                          'Relevance to Core Business': 'relevance'
                                                       },)

df_suzy.head(3)

,company,date_updated_suzy,Main Business,Area of Business Value,relevance,os_engagement,todo_member,todo_adopter,Linux Foundation (Platinum + Gold),GitHub list https://github.com/collections/open-source-organizations,OSCI Index top 60(as of Feb 2021),Reasoning,GitHub,Note,Response to survey,Survey: What do you consider to be your company's dominant open source investment where there are likely shared dependencies?,Survey: Want to hear more?,Emailed to:,in_suzy
0,AT&T,5/7/21,Telcom/operator is main revenue source (also m...,software,Accelerant,Co-Leading,NaN,NaN,X,NaN,NaN,I sense that AT&T sees investments as strategi...,NaN,NaN,NaN,NaN,NaN,NaN,1
1,SAIC,5/7/21,SAIC integrates multiple disciplines to provid...,services,Accelerant,Combining,NaN,NaN,NaN,NaN,NaN,SAIC gov't focused. They contribute (e.g. RedH...,https://github.com/saic-oss,NaN,NaN,NaN,NaN,NaN,1
2,Vapor IO,5/7/21,edge computing infra,software,Accelerant,Co-Leading,NaN,NaN,NaN,NaN,NaN,"Using os for trust, function, ecosystem develo...",NaN,NaN,NaN,NaN,NaN,NaN,1


In [36]:
keep_cols = ['company', 'area','os_engagement', 'in_suzy', 'relevance' 
            ]
            #'activeContributors', 'activeContributorsChange', 
            # 'totalCommunity', 'totalCommunityChange', 'position', 'industry'
            #]

df_suzy = df_suzy.filter(keep_cols)

print(df_suzy.shape)
df_suzy.head(3)

(141, 4)


,company,os_engagement,in_suzy,relevance
0,AT&T,Co-Leading,1,Accelerant
1,SAIC,Combining,1,Accelerant
2,Vapor IO,Co-Leading,1,Accelerant


In [37]:
print(len(df_suzy['company'].unique()))

141


In [38]:
df_check = df_suzy.groupby(['in_suzy']).count()
df_check

,company,os_engagement,relevance
in_suzy,,,
1,141,55,55


## Merge: S&P 500 + OSCI + Suzy

In [39]:
print(df_parta.shape)
print(len(df_parta['company'].unique()))

print(df_suzy.shape)
print(len(df_suzy['company'].unique()))

(781, 8)
781
(141, 4)
141


In [40]:
df_partb = df_sorted.merge(df_suzy, left_on="company", right_on="company", how="outer")
df_partb['company_count']=1
print(df_partb.shape)
df_partb.head()

(875, 12)


,symbol,company,sector,country,osci_industry,in_sp500,in_osci,osci_position,os_engagement,in_suzy,relevance,company_count
0,MMM,3M,Industrials,United States,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
1,NaN,4teamwork,NaN,NaN,Technology,NaN,1.0,285.0,NaN,NaN,NaN,1
2,NaN,5minds,NaN,NaN,Technology,NaN,1.0,276.0,NaN,NaN,NaN,1
3,AOS,A. O. Smith,Industrials,United States,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
4,ADM,ADM,Consumer Staples,United States,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1


In [41]:
print(len(df_partb['company'].unique()))

875


In [42]:
df_check = df_partb.groupby(['in_osci','in_sp500','in_suzy','company_count']).count()
df_check

,,,,symbol,company,sector,country,osci_industry,osci_position,os_engagement,relevance
in_osci,in_sp500,in_suzy,company_count,,,,,,,,
1.0,1.0,1.0,1,9,9,9,9,9,9,3,3


## 4. Data Source:  OSPO Landscape(ToDo Group)
https://landscape.todogroup.org

In [43]:
file_name = 'ospo_todo_landscape.csv'

df_ospo_all = pd.read_csv('data_raw/'+file_name)
df_ospo_all['in_ospo']=1

print(df_ospo_all.shape)
df_ospo_all.head(3)

(225, 41)


,Name,Organization,Homepage,Logo,Twitter,Crunchbase URL,Market Cap,Ticker,Funding,Member,Relation,License,Headquarters,Latest Tweet Date,Description,Crunchbase Description,Crunchbase Homepage,Crunchbase City,Crunchbase Region,Crunchbase Country,Crunchbase Twitter,Crunchbase Linkedin,Crunchbase Ticker,Crunchbase Kind,Crunchbase Min Employees,Crunchbase Max Employees,Category,Subcategory,OSS,Github Repo,Github Stars,Github Description,Github Latest Commit Date,Github Latest Commit Link,Github Release Date,Github Release Link,Github Start Commit Date,Github Start Commit Link,Github Contributors Count,Github Contributors Link,in_ospo
0,Adobe (Member),Adobe,https://www.adobe.com/,https://landscape.todogroup.org/logos/adobe-me...,https://twitter.com/Adobe,https://www.crunchbase.com/organization/adobe,1.887401e+11,ADBE,2.500000e+06,General,member,NotOpenSource,"San Jose, California",2022-05-04 13:00:38,Adobe is a software company that provides its ...,Adobe is a software company that provides its ...,http://www.adobe.com,San Jose,California,United States,http://twitter.com/Adobe,https://www.linkedin.com/company/adobe,ADBE,market_cap,10001.0,1000000.0,TODO Group Member,General,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Aiven (Member),Aiven,https://github.com/aiven/,https://landscape.todogroup.org/logos/aiven-me...,https://twitter.com/aiven_io,https://www.crunchbase.com/organization/aiven,NaN,NaN,2.100000e+08,General,member,NotOpenSource,"Helsinki, Finland",2022-05-04 11:33:18,Aiven is an information technology company tha...,Aiven is an information technology company tha...,https://aiven.io,Helsinki,Southern Finland,Finland,https://twitter.com/aiven_io,https://www.linkedin.com/company/aiven/,NaN,funding,251.0,500.0,TODO Group Member,General,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Apple (Member),Apple,https://opensource.apple.com/,https://landscape.todogroup.org/logos/apple-me...,https://twitter.com/apple,https://www.crunchbase.com/organization/apple,2.608140e+12,AAPL,6.150230e+09,General,member,NotOpenSource,"Cupertino, California",NaN,Apple is a multinational corporation that desi...,Apple is a multinational corporation that desi...,http://www.apple.com,Cupertino,California,United States,https://twitter.com/apple,https://www.linkedin.com/company/apple,AAPL,market_cap,10001.0,1000000.0,TODO Group Member,General,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [44]:
df_ospo_all = df_ospo_all.rename(columns=
                         {'Name': 'ospo_status',
                          'Organization': 'company',
                          'Ticker': 'symbol_ospo',
                          'Market Cap': 'market_cap',
                          'Crunchbase Country': 'country',
                          'License': 'license'
                                                        },)

keep_cols = ['ospo_status','company', 'symbol_ospo','market_cap', 'country', 'relevance',
             'in_ospo'
            ]

df_ospo_all = df_ospo_all.filter(keep_cols)



print(df_ospo_all.shape)
df_ospo_all.head(3)

(225, 6)


,ospo_status,company,symbol_ospo,market_cap,country,in_ospo
0,Adobe (Member),Adobe,ADBE,1.887401e+11,United States,1
1,Aiven (Member),Aiven,NaN,NaN,Finland,1
2,Apple (Member),Apple,AAPL,2.608140e+12,United States,1


In [45]:
#strings = ['member', 'Member']
#output = df_ospo['ospo_status'].isin(strings)

output = df_ospo_all['ospo_status'].str.contains('adopter',regex=True)

df_ospo = df_ospo_all.loc[df_ospo_all['ospo_status'].str.contains("adopter", case=False)]

print(df_ospo.shape)
df_ospo.head(3)

(102, 6)


,ospo_status,company,symbol_ospo,market_cap,country,in_ospo
89,Adobe (Adopter),Adobe,ADBE,1.887401e+11,United States,1
90,Aiven (Adopter),Aiven,NaN,NaN,Finland,1
91,Alibaba (Adopter),Alibaba Group,9988.HK,2.692284e+11,China,1


In [46]:
df_sample = df_ospo.sample(5)
df_sample

,ospo_status,company,symbol_ospo,market_cap,country,in_ospo
91,Alibaba (Adopter),Alibaba Group,9988.HK,2.692284e+11,China,1
179,Uber (Adopter),Uber,UBER,5.133725e+10,United States,1
132,Google (Adopter),Google,GOOG,1.524992e+12,United States,1
159,Qualcomm (Adopter),Qualcomm,QCOM,1.614052e+11,United States,1
122,EPAM (Adopter),EPAM Systems,EPAM,1.681204e+10,United States,1


In [47]:
print(len(df_ospo['company'].unique()))

101


Note: 
- dataset has 102 rows
- there are 2 records for GitLab (one for US, China)

In [48]:
#df_ospo

In [49]:
df_check = df_ospo.groupby(['in_ospo']).count()
df_check

,ospo_status,company,symbol_ospo,market_cap,country
in_ospo,,,,,
1,102,102,67,67,102


## Merge: S&P 500 + OSCI + Suzy + OSPO


In [50]:
print(df_partb.shape)
print(len(df_partb['company'].unique()))

print(df_ospo.shape)
print(len(df_ospo['company'].unique()))

(875, 12)
875
(102, 6)
101


In [51]:
# Merge on 'company' only

df_all = df_partb.merge(df_ospo, left_on="company", right_on="company", how="outer")
# df_all['company_count']=1
# print(df_all.shape)
# df_all.head()

In [52]:
# Merge on: 'company', 'country'

#df_all = df_partb.merge(df_ospo, left_on=["company", "country"], right_on=["company", "country"], how="outer")
df_all['company_count']=1
print(df_all.shape)
df_all.head()

(918, 17)


,symbol,company,sector,country_x,osci_industry,in_sp500,in_osci,osci_position,os_engagement,in_suzy,relevance,company_count,ospo_status,symbol_ospo,market_cap,country_y,in_ospo
0,MMM,3M,Industrials,United States,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
1,NaN,4teamwork,NaN,NaN,Technology,NaN,1.0,285.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
2,NaN,5minds,NaN,NaN,Technology,NaN,1.0,276.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3,AOS,A. O. Smith,Industrials,United States,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
4,ADM,ADM,Consumer Staples,United States,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


In [53]:
# reorder columns
new_order = ['symbol', 'company', 'country_x','country','sector', 'osci_industry', 'in_sp500', 'in_ospo', 'in_osci', 'in_suzy', 'osci_position',
            'os_engagement','relevance','company_count','ospo_status', 'symbol_ospo','market_cap'
             ]
df_all = df_all.reindex(new_order, axis=1)

df_all.head(2)

,symbol,company,country_x,country,sector,osci_industry,in_sp500,in_ospo,in_osci,in_suzy,osci_position,os_engagement,relevance,company_count,ospo_status,symbol_ospo,market_cap
0,MMM,3M,United States,NaN,Industrials,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
1,NaN,4teamwork,NaN,NaN,NaN,Technology,NaN,NaN,1.0,NaN,285.0,NaN,NaN,1,NaN,NaN,NaN


## Clean data

In [54]:
# If in S&P 500 and country is missing, make it "United States"

#df_all['country'].mask(df_all['company'] == '5minds', 'Germany', inplace=True)

# if in_sp500=1 and country is missing, fill it in with "United States"
# By checking multiple conditions

# df_new = df_all.copy()

# df_new['country'] = np.where((df_new['in_sp500'] == 1) & (df_new['country'] == " "), "United States", df_new['country'])
# df_new.head(2)

# conditions = [
#     df_all['in_sp500'].eq(1) & df_all['country'].eq("")]

# choices = ["United States"]

# df_all['country'] = np.select(conditions, choices, default="")


In [55]:
#print x


In [56]:
df_all['country_x'].mask(df_all['company'] == '4teamwork', 'Switzerland', inplace=True)
df_all['country_x'].mask(df_all['company'] == '5minds', 'Germany', inplace=True)



df_all['listed'] = "xxx"

df_all['symbol'].mask(df_all['company'] == 'AMD', 'AMD', inplace=True)
df_all['listed'].mask(df_all['company'] == 'AMD', 'NASDAQ', inplace=True)
df_all['country_x'].mask(df_all['company'] == 'AMD', 'United States', inplace=True)

df_all['symbol'].mask(df_all['company'] == 'ARM', 'ARM', inplace=True)
df_all['listed'].mask(df_all['company'] == 'ARM', 'London SE', inplace=True)
df_all['country_x'].mask(df_all['company'] == 'ARM', 'United Kingdom', inplace=True)
df_all['sector'].mask(df_all['company'] == 'ARM', 'Technology', inplace=True)


df_all['symbol'].mask(df_all['company'] == 'Abilian', 'Private', inplace=True)
df_all['listed'].mask(df_all['company'] == 'Abilian', 'Private', inplace=True)
df_all['country_x'].mask(df_all['company'] == 'Abilian', 'France', inplace=True)
df_all['sector'].mask(df_all['company'] == 'Abilian', 'Technology', inplace=True)

df_all['symbol'].mask(df_all['company'] == 'Acsone', 'Private', inplace=True)
df_all['listed'].mask(df_all['company'] == 'Acsone', 'Private', inplace=True)
df_all['country_x'].mask(df_all['company'] == 'Acsone', 'Belgium', inplace=True)
df_all['sector'].mask(df_all['company'] == 'Acsone', 'Technology', inplace=True)

# https://adhocteam.us/
#https://www.linkedin.com/company/ad-hoc-llc/
df_all['symbol'].mask(df_all['company'] == 'Ad Hoc', 'Private', inplace=True)
df_all['listed'].mask(df_all['company'] == 'Ad Hoc', 'Private', inplace=True)
df_all['country_x'].mask(df_all['company'] == 'Ad Hoc', 'United States', inplace=True)
df_all['sector'].mask(df_all['company'] == 'Ad Hoc', 'Technology', inplace=True)

# https://adguard.com/en/welcome.html
# https://www.linkedin.com/company/adguard/about/
df_all['symbol'].mask(df_all['company'] == 'Adguard', 'Private', inplace=True)
df_all['listed'].mask(df_all['company'] == 'Adguard', 'Private', inplace=True)
df_all['country_x'].mask(df_all['company'] == 'Adguard', 'United States', inplace=True)
df_all['sector'].mask(df_all['company'] == 'Adguard', 'Technology', inplace=True)



---

## Save `df` to `csv` file

In [57]:
print(df_all.shape)
print(len(df_all['company'].unique()))

(918, 18)
917


In [58]:
df_all.to_csv('data_derived/focused.csv', index=False)

---

## Checks

In [59]:
df_all.shape

(918, 18)

In [63]:
df_all['country_x'].unique()

array(['United States', 'Switzerland', 'Germany', 'United Kingdom',
       'France', 'Belgium', nan], dtype=object)

In [64]:
# have country info for 585/956
df_all['country_x'].value_counts(normalize=False)

United States     508
Switzerland         1
Germany             1
United Kingdom      1
France              1
Belgium             1
Name: country_x, dtype: int64

In [62]:
print x


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(x)? (3872341986.py, line 1)

In [ ]:
df_check = df_all.groupby(['in_sp500', 'in_osci', 'in_suzy', 'in_ospo','company_count']).count()
df_check

In [ ]:
df_all.head(5)

In [ ]:
# reorder columns
new_order = ['symbol', 'company', 'sector', 'osci_industry', 'in_sp500', 'in_ospo', 'in_osci', 'in_suzy', 'osci_position',
            'os_engagement','relevance','company_count','ospo_status', 'symbol_ospo','market_cap','country'
             ]
df_new = df_all.reindex(new_order, axis=1)

df_new.head(2)

In [ ]:
#df_check = df_all.groupby(['in_sp500']).count()
#df_check

In [ ]:
# if in_sp500=1 and country is missing, fill it in with "United States"
# By checking multiple conditions
#df_new['country'] = np.where((df_new['in_sp500'] == 1) & (df_new['country'] == " "), "United States", df_new['country'])
#df_new.head(2)

# NEXT STEPS
- sort and check company names
- add in missing country names
- Watch Lux video:  https://youtu.be/O28RZ8SsSQk
- Run Lux examples with this dataset:  

In [ ]:
print("Today's date:", current_date)

# END OF CODE

# END OF CODE

# END OF CODE

---

In [ ]:
print x

### Lux video

#### Example hands-on Lux tutorial
https://mybinder.org/v2/gh/lux-org/lux-binder/master?urlpath=lab/tree/demo/hpi_covid_demo.ipynb


In [ ]:
import pandas as pd
#df = pd.read_csv("data.csv")
import lux

df = df_all.copy()
#df.groupby(" ").mean()["xxx"]

In [ ]:
df

In [ ]:
# numerical variables
df.intent = ['market_cap','osci_position']

In [ ]:
df

In [ ]:
# for 1-d series, such as time, we can see patterns of other variables over time
# Example covid['Day']


In [ ]:
df['osci_position']

In [ ]:
# Quick-and-dirty experimentation with visualizations

from lux.vis.Vis import Vis
Vis(['osci_position'], df)

In [ ]:
df_meetup['city'].mask(df_meetup['city'] == 'Raleigh', 'Raleigh-Durham', inplace=True)



In [ ]:
# rename columns
df_meetup.rename(columns={"name": "meetup_name", 
                          "members": "meetup_members",
                          "fullurl": "meetup_url",
                          "created": "meetup_created",
                          "status": "meetup_status"
                            },
                   inplace = True,
                   errors="raise")

## 2. Data Source:  Chapter List
### Get raw data

In [ ]:
!ls -alt data_raw/

In [ ]:
file_name = '1_pyladies_export_2021_03_01.csv'

chapter_list = pd.read_csv(datapath+file_name)  

In [ ]:
chapter_list.head()

In [ ]:
chapter_list.info()

In [ ]:
chapter_list.shape

In [ ]:
df_chapter = chapter_list.copy()
# rename columns
df_chapter.rename(columns={"First Name [Required]": "city1", 
                             "Last Name [Required]": "city2", 
                             "Email Address [Required]": "email",
                             "Last Sign In [READ ONLY]": "email_signin_date",
                             "Email Usage [READ ONLY]": "email_usage"
                            },
                   inplace = True,
                   errors="raise")


# delete columns:  'active' is same for all
df_chapter = df_chapter.drop(['Status [READ ONLY]'], axis=1)

# fix data issue
df_chapter['city2'].mask(df_chapter['city2'] == \
                      'PyLladies', \
                      'PyLadies', inplace=True)

df_chapter['city2'].mask(df_chapter['city2'] == \
                      'PyLdies', \
                      'PyLadies', inplace=True)

# swap some rows
df_chapter['city_temp1'] = df_chapter['city1'].str.upper() 
df_chapter['city_temp2'] = df_chapter['city2'].str.upper() 

df_chapter['city'] = df_chapter['city1']
df_chapter['temp'] = df_chapter['city2']

df_chapter['city'].mask(df_chapter['city_temp1'] == 'PYLADIES', df_chapter['city2'], inplace=True)

df_chapter['temp'].mask(df_chapter['city_temp2'] == 'PYLADIES', df_chapter['city1'], inplace=True)

# exclude some rows
df_chapter = df_chapter[(df_chapter['city_temp1'] == 'PYLADIES') | (df_chapter['city_temp2'] == 'PYLADIES')] 

df_chapter = df_chapter[(df_chapter['city1'] != 'Admin')] 
df_chapter = df_chapter[(df_chapter['city2'] != 'Sponsorship')] 
df_chapter = df_chapter[(df_chapter['city2'] != 'Info')] 
df_chapter = df_chapter[(df_chapter['city2'] != 'Systems')] 
df_chapter = df_chapter[(df_chapter['city1'] != 'Communications')] 

# clean up city names
df_chapter['city'].mask(df_chapter['city'] == 'NYC', 'New York', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Princeton NJ', 'Princeton', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Pune, India', 'Pune', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'TC', 'Minneapolis', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Goa', 'Panaji Goa', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Visakhapatnam/Andhra Pradesh/India', 'Visakhapatnam', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Washington', 'Washington DC', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Aba / Abia State', 'Aba', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'West Windsor NJ', 'West Windsor', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Medellin', 'Medellín', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Albania', 'Tirania', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'BH', 'Belo Horizonte', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Guatemala', 'Guatemala City', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Inland Empire', 'Riverside', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Vale', 'Vale do Paraiba', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Plateau', 'Jos Plateau', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Recife, Brasil', 'Recife', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Rio de Janerio', 'Rio de Janeirio', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Salt Lake', 'Salt Lake City', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'SA', 'San Antonio', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'São Carlos, Brasil', 'São Carlos', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Taiwan', 'Taipei', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Milan', 'Milano', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Munich', 'München', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Rio de Janeirio', 'Rio de Janeiro', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Bangalore', 'Bengaluru', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'DC', 'Washington DC', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'DF', 'Brasilia', inplace=True)
df_chapter['city'].mask(df_chapter['city'] == 'Congo-Brazzaville', 'Brazzaville', inplace=True)


import datetime
df_chapter['date_logged'] = df_chapter['email_signin_date']

df_chapter['email_signin_date'].mask(df_chapter['email_signin_date'] == 'Never logged in', 'NaN', inplace=True)

df_chapter['email_signin_date'] = df_chapter['email_signin_date'].astype('datetime64[ns]').dt.date

# drop temp variables
df_chapter = df_chapter.drop(['city1','city2','city_temp1','city_temp2','temp'], axis=1)

df_chapter['registered_db'] = 1

In [ ]:
df_chapter.head(300)

In [ ]:
df_chapter.shape

In [ ]:
df_all = pd.merge(df_chapter, df_meetup, on=['city'],  how='outer')

In [ ]:
print('emails (all):      ', chapter_list.shape)
print('emails (chapters): ', df_chapter.shape)
print('-'*30)
print('meetup dashboard:  ', df_meetup.shape)
print('-'*30)
print('Merged:            ', df_all.shape)

In [ ]:
df_all.sort_values(by=['city'])

In [ ]:
df_all[['email','email_signin_date','email_usage','city','country','region', 'registered_db','meetup_db','meetup_status','last_event']]

In [ ]:
df_all = df_all.drop(['country','region'], axis=1)

## 3. Data Source:  Reshama's Inventory
### Get raw data

In [ ]:
file_name3 = '2021_PyLadies_Inventory - 2021-03.csv'
df_inventory = pd.read_csv(datapath+file_name3)  
df_inventory['inv_db'] = 1

In [ ]:
#df_inventory['city'].mask(df_inventory['city'] == 'Caxias Do Sul', 'Caxias do Sul', inplace=True)
#df_inventory['city'].mask(df_inventory['city'] == 'Duque De Caxias', 'Duque de Caxias', inplace=True)
#df_inventory['city'].mask(df_inventory['city'] == 'Salt Lake Area', 'Salt Lake City', inplace=True)


In [ ]:
df_inventory.head(5)

In [ ]:
df_inventory.shape

In [ ]:
df_use = pd.merge(df_all, df_inventory, on=['city'],  how='outer')

In [ ]:
df_use = df_use.sort_values(by=['city'])

df_use.head(4)

In [ ]:
# As a default, if meetup group exists, use that member count

#df_test=df_use.copy()

# df_test[['city', 'country','region', \
#         'registered_db','meetup_db','main_platform','meetup_members','members_followers']].head(10)
# df_test['members_followers'].mask(df_test['meetup_members'] == 'NaN', df_test['members_followers'], inplace=True)


df_use['members_followers']=df_use['meetup_members'].mask(pd.isnull, df_use['members_manual'])


df_use[['city', 'country','region', \
        'registered_db','meetup_db','main_platform','meetup_members','members_manual','members_followers']].head(100)


In [ ]:
df_use[['city', 'country','region', \
        'registered_db','meetup_db','main_platform','meetup_members','members_followers']]


In [ ]:
df_use.shape

In [ ]:
#df_use[['email','email_signin_date','email_usage','city','country_x','region_x', 'country_y','region_y', 'registered_db','meetup_db','status','last_event']]
df_use[['email','email_signin_date','city', 'country','region', \
        'registered_db','meetup_db','main_platform']]


In [ ]:
df_use['chapter_count'] = 1

In [ ]:
df_use['meetup_db'].unique()

In [ ]:
df_use['meetup_db'].value_counts(normalize=True)

In [ ]:
df_use['meetup_db'].value_counts(normalize=False)

In [ ]:
#df_use.head(3)

In [ ]:
df_use.shape

### Remove Duplicates

In [ ]:
# get names of indexes for which 

# Note:  There are 2 registered chapters for Bengaluru
index_names = df_use[ df_use['email'] == 'bangalore@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'sa@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'mum@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'newark@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'alb@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'tampa@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

index_names = df_use[ df_use['email'] == 'tunisia@pyladies.com' ].index 
df_use.drop(index_names, inplace = True) 

# Note:  There are 2 meetup groups with same name
index_names = df_use[ df_use['meetup_url'] == 'https://www.meetup.com/Pyladies-Bogota/'].index 
df_use.drop(index_names, inplace = True) 

# Note:  There are 2 meetup groups with same name
index_names = df_use[ df_use['meetup_url'] == 'https://www.meetup.com/PyLadies-PUC-Rio/'].index 
df_use.drop(index_names, inplace = True) 


df_use.shape

### Create `location` column

In [ ]:
df_use['location'] = df_use['city'].astype(str) + ' ' + df_use['country'].astype(str)
df_use['location2']= df_use['city'].astype(str) \
                        + ' ' \
                        + df_use['state-province'].astype(str) \
                        + ' ' \
                        + df_use['country'].astype(str)


subset = df_use[['location','location2', 'city', 'state-province','country','country_level','region', 'main_platform']]
subset.head(300)

In [ ]:
df_use.head()

## For Venn Diagram

In [ ]:
# subset = df_use[df_use["meetup"].isin(["1"])]
# subset

In [ ]:
df_use[['email','email_signin_date','city', 'country','region', \
        'registered_db','meetup_db','main_platform']]

In [ ]:
df_use_check=df_use[['meetup_db','registered_db','inv_db','chapter_count']]

In [ ]:
df_check = df_use_check.groupby(['meetup_db']).sum()

In [ ]:
df_check

In [ ]:
df_check = df_use_check.groupby(['registered_db']).sum()

In [ ]:
df_check

In [ ]:
df_check = df_use_check.groupby(['inv_db']).sum()

In [ ]:
df_check

In [ ]:
df_check = df_use.groupby(['meetup_db','registered_db','inv_db']).sum()
#df_check = df_use.groupby(['meetup_db','registered_db','inv_db']).count()
df_check

In [ ]:
df_use[['meetup_db','registered_db','inv_db','email','email_signin_date','city', 'state-province','country','region', 'main_platform']]

In [ ]:
df_use.shape

In [ ]:
# subset = df_use.query('meetup_db' in(1))
# #& registered_db == '1')


# temp = df_use[df_use.eval("meetup_db = '1'")]


In [ ]:
df_use['status_derived'].value_counts(normalize=False)

In [ ]:
df_use['main_platform'].value_counts(normalize=False)

In [ ]:
subset = df_use[df_use["country_level"].isin(["1"])]
print(subset.shape)
subset.head()


In [ ]:
df_use['region'].value_counts(normalize=False)

In [ ]:
df_use['members_followers'].sum()

In [ ]:
(df_use['country'].unique()).shape

In [ ]:
df_use['country'].value_counts(normalize=False)[:10]

In [ ]:
# 75 meetup
# 60 not found
# 135
# 261 - 135 = 125 -45 Brazil = 90



# Set cvs file

In [ ]:
#df_use.sort_values(by=['region','country','city','state-province'])

df_use.sort_values(['region','country','city','state-province'], inplace=True, ascending=True)


In [ ]:
df_use.to_csv('data_derived/pyladies_clean.csv',
              columns=['region','country','city','state-province','email','email_signin_date','email_usage', \
                       'main_platform','founded','meetup_created','main_platform_link','members_followers', \
                       'status_derived','members_manual','meetup_members', 'meetup_url', 'meetup_status','last_event',\
                       'past_events','upcoming_events','country_level','last_activity', 'facebook_page',\
                       'website','meetup_link', 'twitter_link','instagram_link','other_social_media', \
                       'meetup_db','registered_db','inv_db','meetup_name','location','location2','chapter_count'
                                 ])


In [ ]:
#df_use.info()

In [ ]:
df_use.head()